In [1]:
from mlaut.analyze_results import AnalyseResults
from mlaut.data import Data
import pandas as pd
from mlaut.estimators.estimators import instantiate_default_estimators
from mlaut.analyze_results.scores import ScoreAccuracy

import matplotlib.pyplot as plt
pd.options.display.max_rows = 1000

/media/viktor/Data/PhD/mlaut/py36/local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
data = Data()
input_io = data.open_hdf5('data/openml.h5', mode='r')
out_io = data.open_hdf5('data/openml-classification.h5', mode='a')
analyze = AnalyseResults(hdf5_output_io=out_io, 
                        hdf5_input_io=input_io, 
                        input_h5_original_datasets_group='openml/', 
                        output_h5_predictions_group='experiments/predictions/')


### All datasets

In [3]:
score_accuracy = ScoreAccuracy()

# (errors_per_estimator, 
#  errors_per_dataset_per_estimator) = analyze.prediction_errors(score_accuracy)
 
(errors_per_estimator, 
 errors_per_dataset_per_estimator, 
 errors_per_dataset_per_estimator_df) = analyze.prediction_errors(score_accuracy)

#### Simple average and standard error

In [24]:
avg_and_std_error = analyze.average_and_std_error(errors_per_estimator)
avg_and_std_error

,avg,std_error
K_Means,0.412214,0.010614
BaselineClassifier,0.536248,0.008504
PassiveAggressiveClassifier,0.559298,0.010893
GaussianNaiveBayes,0.567017,0.011205
GradientBoostingClassifier,0.571340,0.011215
BaggingClassifier,0.575503,0.011307
RandomForestClassifier,0.581803,0.011308
BernoulliNaiveBayes,0.586203,0.010751
NeuralNetworkDeepClassifier,0.594388,0.010582
SVC,0.610982,0.009809


#### Average Rank

In [27]:
avg_rank = analyze.ranks(errors_per_estimator, ascending=False)
avg_rank

,avg_rank
RandomForestClassifier,4.829848
SVC,4.852662
BernoulliNaiveBayes,4.912548
BaggingClassifier,4.973384
NeuralNetworkDeepClassifier,5.127376
GaussianNaiveBayes,5.284221
PassiveAggressiveClassifier,5.342205
GradientBoostingClassifier,5.472433
BaselineClassifier,6.944867
K_Means,7.260456


#### Cohen's d

In [6]:
cohens_d = analyze.cohens_d(errors_per_estimator)
cohens_d

,Cohen's d
K_Means-SVC,0.848041
K_Means-NeuralNetworkDeepClassifier,0.749476
BernoulliNaiveBayes-K_Means,-0.710127
K_Means-RandomForestClassifier,0.674260
BaggingClassifier-K_Means,-0.649237
GradientBoostingClassifier-K_Means,-0.635441
GaussianNaiveBayes-K_Means,-0.618456
K_Means-PassiveAggressiveClassifier,0.596329
BaselineClassifier-K_Means,-0.562330
BaselineClassifier-SVC,0.354973


#### t-test

In [7]:
t_test, t_test_df = analyze.t_test(errors_per_estimator)
t_test_df

,pair,t_statistic,p_value
0,BaggingClassifier - BaselineClassifier,2.771919,5.671213e-03
1,BaggingClassifier - BernoulliNaiveBayes,-0.685128,4.934147e-01
2,BaggingClassifier - GaussianNaiveBayes,0.532581,5.944367e-01
3,BaggingClassifier - GradientBoostingClassifier,0.261157,7.940229e-01
4,BaggingClassifier - K_Means,10.518846,1.134398e-24
5,BaggingClassifier - NeuralNetworkDeepClassifier,-1.218284,2.233896e-01
6,BaggingClassifier - PassiveAggressiveClassifier,1.031179,3.026946e-01
7,BaggingClassifier - RandomForestClassifier,-0.393585,6.939671e-01
8,BaggingClassifier - SVC,-2.367937,1.806796e-02
9,BaselineClassifier - BernoulliNaiveBayes,-3.640793,2.850537e-04


#### sign test

In [8]:
sign_test, sign_test_df = analyze.sign_test(errors_per_estimator)
sign_test_df

,pair,t_statistic,p_value
0,BaggingClassifier - BaselineClassifier,3.278670,1.042977e-03
1,BaggingClassifier - BernoulliNaiveBayes,-0.578004,5.632616e-01
2,BaggingClassifier - GaussianNaiveBayes,0.503013,6.149548e-01
3,BaggingClassifier - GradientBoostingClassifier,0.283319,7.769321e-01
4,BaggingClassifier - K_Means,9.894966,4.377590e-23
5,BaggingClassifier - NeuralNetworkDeepClassifier,-1.119071,2.631101e-01
6,BaggingClassifier - PassiveAggressiveClassifier,1.111764,2.662395e-01
7,BaggingClassifier - RandomForestClassifier,-0.382156,7.023454e-01
8,BaggingClassifier - SVC,-1.989831,4.660958e-02
9,BaselineClassifier - BernoulliNaiveBayes,-4.668882,3.028427e-06


#### t-test with Bonferroni correction

In [9]:
t_test_bonferroni, t_test_bonferroni_df = analyze.t_test_with_bonferroni_correction(errors_per_estimator)
t_test_bonferroni_df

,pair,p_value
0,BaggingClassifier - BaselineClassifier,2.552046e-01
1,BaggingClassifier - BernoulliNaiveBayes,1.000000e+00
2,BaggingClassifier - GaussianNaiveBayes,1.000000e+00
3,BaggingClassifier - GradientBoostingClassifier,1.000000e+00
4,BaggingClassifier - K_Means,5.104792e-23
5,BaggingClassifier - NeuralNetworkDeepClassifier,1.000000e+00
6,BaggingClassifier - PassiveAggressiveClassifier,1.000000e+00
7,BaggingClassifier - RandomForestClassifier,1.000000e+00
8,BaggingClassifier - SVC,8.130582e-01
9,BaselineClassifier - BernoulliNaiveBayes,1.282742e-02


#### Wilcoxon test

In [10]:
wilcoxon_test, wilcoxon_test_df = analyze.wilcoxon_test(errors_per_estimator)
wilcoxon_test_df

,pair,statistic,p_value
0,BaggingClassifier - BaselineClassifier,48916.5,7.981088e-08
1,BaggingClassifier - BernoulliNaiveBayes,36078.0,7.502815e-01
2,BaggingClassifier - GaussianNaiveBayes,37150.0,1.722447e-02
3,BaggingClassifier - GradientBoostingClassifier,19724.5,3.236654e-04
4,BaggingClassifier - K_Means,21593.5,2.684341e-29
5,BaggingClassifier - NeuralNetworkDeepClassifier,42132.0,7.362613e-01
6,BaggingClassifier - PassiveAggressiveClassifier,38599.0,6.211165e-02
7,BaggingClassifier - RandomForestClassifier,16322.5,1.030145e-01
8,BaggingClassifier - SVC,29475.5,4.269111e-02
9,BaselineClassifier - BernoulliNaiveBayes,40632.0,3.382746e-14


#### Friedman test

In [11]:
friedman_test, friedman_test_df = analyze.friedman_test(errors_per_estimator)
friedman_test_df

,statistic,p_value
0,447.92709,7.955256e-91


In [30]:
nemeniy_test = analyze.nemenyi(errors_per_estimator)
nemeniy_test_df = pd.DataFrame(nemeniy_test)
nemeniy_test_df

,BaggingClassifier,BaselineClassifier,BernoulliNaiveBayes,GaussianNaiveBayes,GradientBoostingClassifier,K_Means,NeuralNetworkDeepClassifier,PassiveAggressiveClassifier,RandomForestClassifier,SVC
BaggingClassifier,-1.000000e+00,2.717684e-01,9.998654e-01,9.999980e-01,1.000000e+00,2.530063e-19,9.978357e-01,9.989503e-01,9.999997e-01,8.816357e-01
BaselineClassifier,2.717684e-01,-1.000000e+00,4.327028e-02,5.448475e-01,3.638425e-01,8.617162e-08,1.604297e-02,8.305850e-01,1.243701e-01,5.416324e-04
BernoulliNaiveBayes,9.998654e-01,4.327028e-02,-1.000000e+00,9.937089e-01,9.992784e-01,4.632699e-23,1.000000e+00,9.304125e-01,9.999996e-01,9.964340e-01
GaussianNaiveBayes,9.999980e-01,5.448475e-01,9.937089e-01,-1.000000e+00,1.000000e+00,3.394648e-17,9.687459e-01,9.999957e-01,9.997061e-01,6.522507e-01
GradientBoostingClassifier,1.000000e+00,3.638425e-01,9.992784e-01,1.000000e+00,-1.000000e+00,1.573628e-18,9.932215e-01,9.997840e-01,9.999929e-01,8.119497e-01
K_Means,2.530063e-19,8.617162e-08,4.632699e-23,3.394648e-17,1.573628e-18,-1.000000e+00,1.157278e-24,5.213585e-15,4.263889e-21,2.928280e-29
NeuralNetworkDeepClassifier,9.978357e-01,1.604297e-02,1.000000e+00,9.687459e-01,9.932215e-01,1.157278e-24,-1.000000e+00,8.239855e-01,9.999343e-01,9.997171e-01
PassiveAggressiveClassifier,9.989503e-01,8.305850e-01,9.304125e-01,9.999957e-01,9.997840e-01,5.213585e-15,8.239855e-01,-1.000000e+00,9.875828e-01,3.403343e-01
RandomForestClassifier,9.999997e-01,1.243701e-01,9.999996e-01,9.997061e-01,9.999929e-01,4.263889e-21,9.999343e-01,9.875828e-01,-1.000000e+00,9.692684e-01
SVC,8.816357e-01,5.416324e-04,9.964340e-01,6.522507e-01,8.119497e-01,2.928280e-29,9.997171e-01,3.403343e-01,9.692684e-01,-1.000000e+00


### Per dataset

In [13]:
errors_per_dataset_per_estimator_df

loss  \
AP_Breast_Colon                                    BaggingClassifier            0.528846   
                                                   BaselineClassifier           0.442308   
                                                   BernoulliNaiveBayes          0.528846   
                                                   GaussianNaiveBayes           0.076923   
                                                   GradientBoostingClassifier   0.528846   
                                                   K_Means                      0.509615   
                                                   NeuralNetworkDeepClassifier  0.471154   
                                                   PassiveAggressiveClassifier  0.528846   
                                                   RandomForestClassifier       0.471154   
                                                   SVC                          0.471154   
AP_Breast_Kidney                                   BaggingClassifier            0.410000   
                                                   BaselineClassifier           0.520000   
                                                   BernoulliNaiveBayes          0.410000   
                                                   GaussianNaiveBayes           0.035000   
                                                   GradientBoostingClassifier   0.590000   
                                                   K_Means                      0.910000   
                                                   NeuralNetworkDeepClassifier  0.590000   
                                                   PassiveAggressiveClassifier  0.590000   
                                                   RandomForestClassifier       0.410000   
                                                   SVC                          0.410000   
AP_Breast_Lung                                     BaggingClassifier            0.717949   
                                                   BaselineClassifier           0.397436   
                                                   BernoulliNaiveBayes          0.282051   
                                                   GaussianNaiveBayes           0.032051   
                                                   GradientBoostingClassifier   0.282051   
                                                   K_Means                      0.461538   
                                                   NeuralNetworkDeepClassifier  0.288462   
                                                   PassiveAggressiveClassifier  0.628205   
                                                   RandomForestClassifier       0.717949   
                                                   SVC                          0.282051   
AP_Breast_Omentum                                  BaggingClassifier            0.784173   
                                                   BaselineClassifier           0.302158   
                                                   BernoulliNaiveBayes          0.215827   
                                                   GaussianNaiveBayes           0.086331   
                                                   GradientBoostingClassifier   0.215827   
                                                   K_Means                      0.532374   
                                                   NeuralNetworkDeepClassifier  0.266187   
                                                   PassiveAggressiveClassifier  0.589928   
                                                   RandomForestClassifier       0.784173   
                                                   SVC                          0.215827   
AP_Breast_Ovary                                    BaggingClassifier            0.324022   
                                                   BaselineClassifier           0.402235   
                                                   BernoulliNaiveBayes          0.324022   
                                    

### Check why loss is zero

In [14]:
_, _, _, y_test = data.load_test_train_dts(hdf5_out=out_io, 
                                                            hdf5_in=input_io, 
                                                            dts_name='zoo', 
                                                            dts_grp_path='/openml')
predictions = out_io.load_predictions_for_dataset('zoo')

In [15]:
import numpy as np
for prediction in predictions:
    diffs = y_test-prediction[1]
    print(f'{prediction[0]} - Number of errors: {np.count_nonzero(diffs)}')

BaggingClassifier - Number of errors: 0
BaselineClassifier - Number of errors: 14
BernoulliNaiveBayes - Number of errors: 0
GaussianNaiveBayes - Number of errors: 0
GradientBoostingClassifier - Number of errors: 0
K_Means - Number of errors: 11
NeuralNetworkDeepClassifier - Number of errors: 3
PassiveAggressiveClassifier - Number of errors: 0
RandomForestClassifier - Number of errors: 0
SVC - Number of errors: 0


#### Bagging

In [16]:
bag_predictions = predictions[0][1]
bag_predictions - y_test

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

#### BernoulliNaiveBayes

In [17]:
bnb_predictions = predictions[2][1]
bnb_predictions - y_test

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

#### GaussianNaiveBayes

In [18]:
gnb_predictions = predictions[2][1]
gnb_predictions - y_test

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

#### Random Forest

In [19]:
rf_predictions = predictions[8][1]
rf_predictions - y_test

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

#### SVM

In [20]:
svc_predictions = predictions[9][1]
svc_predictions-y_test

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

#### Passive Aggressive Classifier

In [21]:
pac_predictions = predictions[8][1]
pac_predictions-y_test

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

## Save tables to $\LaTeX$

In [33]:
#average and standard error
with open('../mlaut_paper/tables/avg_and_st_error.tex', 'w') as tf:
    tf.write(avg_and_std_error.to_latex())
    
#average rank
with open('../mlaut_paper/tables/avg_rank.tex', 'w') as tf:
    tf.write(avg_rank.to_latex())
#Cohen's D
with open('../mlaut_paper/tables/cohens_d.tex', 'w') as tf:
    tf.write(cohens_d.to_latex())
#t-test
with open('../mlaut_paper/tables/t_test.tex', 'w') as tf:
    tf.write(t_test_df.to_latex())
with open('../mlaut_paper/tables/t_test_bonferroni.tex', 'w') as tf:
    tf.write(t_test_bonferroni_df.to_latex())
#sign test
with open('../mlaut_paper/tables/sign_test.tex', 'w') as tf:
    tf.write(sign_test_df.to_latex())
#t-test with Bonferroni correction
with open('../mlaut_paper/tables/t_test_bonferroni.tex', 'w') as tf:
    tf.write(t_test_bonferroni_df.to_latex())
#Wilcoxon
with open('../mlaut_paper/tables/wilxocon_test.tex', 'w') as tf:
    tf.write(wilcoxon_test_df.to_latex())
#Friedman test
with open('../mlaut_paper/tables/friedman_test.tex', 'w') as tf:
    tf.write(friedman_test_df.to_latex())
#Nemeniy test
with open('../mlaut_paper/tables/nemeniy_test.tex', 'w') as tf:
    tf.write(nemeniy_test_df.to_latex())
#Errors per dataset per estimator
with open('../mlaut_paper/tables/errors_per_dataset_per_estimator.tex', 'w') as tf:
    tf.write(errors_per_dataset_per_estimator_df.to_latex())